In [1]:
# Configuração do ambiente
%cd ..  
!cd  

C:\Users\flavi\Documents\GitHub\Projeto_7_Sistema_de_Recomendacao
C:\Users\flavi\Documents\GitHub\Projeto_7_Sistema_de_Recomendacao


In [2]:
# Importações necessárias 
import sys
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from src.config.paths import DADOS_BRUTOS, DADOS_TRATADOS
from src.config.auxiliares_ml import downcast_dataframe

In [3]:
# Importando a base de dados

df = pd.read_csv(DADOS_BRUTOS)

In [4]:
# Observando as primeiras linhas da base 

df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94


In [5]:
# Observando as informações da base 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [6]:
# Observando os valores nulos da base

df.isnull().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64

In [7]:
# Filtrando os valores nulos de 'Postal Code'

df.loc[df['Postal Code'].isnull(), 'Postal Code'] = 5401

In [8]:
# Criando uma função para corrigir os types das variáveis

def ajustar_tipos(df):
    df_corrigido = df.copy()
    
    for col in df_corrigido.columns:
        if col in ['Order Date', 'Ship Date']:
            # Manter essas colunas como tipo data
            df_corrigido[col] = pd.to_datetime(df_corrigido[col], errors='coerce')
        elif pd.api.types.is_numeric_dtype(df_corrigido[col]):
            # Se for numérico, verificar se pode ser convertido para inteiro
            if all(df_corrigido[col].dropna() % 1 == 0):
                df_corrigido[col] = df_corrigido[col].astype(int)
        elif pd.api.types.is_string_dtype(df_corrigido[col]):
            # Se for string e tiver muitas repetições, pode ser categorizado
            unique_ratio = df_corrigido[col].nunique() / len(df_corrigido[col])
            if unique_ratio < 0.5:  # Pode ajustar esse valor conforme necessário
                df_corrigido[col] = df_corrigido[col].astype('category')
        elif pd.api.types.is_object_dtype(df_corrigido[col]):
            # Verificar se é uma data
            try:
                df_corrigido[col] = df_corrigido[col].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') if isinstance(x, str) else x)
            except (ValueError, TypeError):
                pass
    
    return df_corrigido

In [9]:
# Utilizando a função 

df_corrigido = ajustar_tipos(df)
df_corrigido.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode              category
Customer ID            category
Customer Name          category
Segment                category
Country                category
City                   category
State                  category
Postal Code               int64
Region                 category
Product ID             category
Category               category
Sub-Category           category
Product Name           category
Sales                   float64
dtype: object

In [10]:
# Fazendo o downcast da base 

df = downcast_dataframe(df)

In [11]:
# Sanvando a base tratada na extensão .parquet

df.to_parquet(DADOS_TRATADOS)